In [1]:
# coding: UTF-8
import os
import torch
import numpy as np
import pickle as pkl
from tqdm import tqdm
import time
from datetime import timedelta
import time
import torch
import numpy as np
from train_eval import train, init_network
from importlib import import_module
import argparse
dataset = 'Newtrain_qinggan'  # 数据集

# 搜狗新闻:embedding_SougouNews.npz, 腾讯:embedding_Tencent.npz, 随机初始化:random
embedding = 'embedding_Newtrain_300.npz'

model_name = 'TextRCNN'  # 'TextRCNN'  # TextCNN, TextRNN, FastText, TextRCNN, TextRNN_Att, DPCNN, Transformer


x = import_module('models.' + model_name)
config = x.Config(dataset, embedding)
config.pad_size = 64

MAX_VOCAB_SIZE = 20000  # 词表长度限制
UNK, PAD = '<UNK>', '<PAD>'  # 未知字，padding符号
ues_word = True
from utils import get_time_dif, build_vocab, build_dataset2, build_iterator2


vocab, train, test = build_dataset2(config, ues_word)

Vocab size: 12678


8000it [00:00, 69866.49it/s]
2653it [00:00, 70938.98it/s]


In [2]:
import torch.nn.functional as F


train_iter = build_iterator2(train, config)
test_iter = build_iterator2(test, config)
# train_iter.__next__()

In [ ]:
config.n_vocab = len(vocab)
model = x.Model(config).to(config.device)
if model_name != 'Transformer':
    init_network(model)
print(model.parameters)

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from utils import get_time_dif
from tensorboardX import SummaryWriter
def train_qinggan(config, model, train_iter, dev_iter):
    start_time = time.time()
    model.train()
    optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)

    # 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
    # scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    total_batch = 0  # 记录进行到多少batch
    dev_best_loss = float('inf')
    last_improve = 0  # 记录上次验证集loss下降的batch数
    flag = False  # 记录是否很久没有效果提升
    writer = SummaryWriter(log_dir=config.log_path + '/' + time.strftime('%m-%d_%H.%M', time.localtime()))
    for epoch in range(config.num_epochs):
        print('Epoch [{}/{}]'.format(epoch + 1, config.num_epochs))
        # scheduler.step() # 学习率衰减
        for i, (trains, qinggan_label, zhuti_label) in enumerate(train_iter):
            print('Batch: ', i)
            outputs = model(trains)
            model.zero_grad()
            loss = F.cross_entropy(outputs, qinggan_label)
            loss.backward()
            optimizer.step()
            if total_batch % 2 == 0:
                # 每多少轮输出在训练集和验证集上的效果
                true = qinggan_label.data.cpu()
                predic = torch.max(outputs.data, 1)[1].cpu()
                train_acc = metrics.accuracy_score(true, predic)
                dev_acc, dev_loss = evaluate(config, model, dev_iter)
                if dev_loss < dev_best_loss:
                    dev_best_loss = dev_loss
                    torch.save(model.state_dict(), config.save_path)
                    improve = '*improve'
                    last_improve = total_batch
                else:
                    improve = ''
                time_dif = get_time_dif(start_time)
                msg = 'Iter: {0:>6},  Train Loss: {1:>5.2},  Train Acc: {2:>6.2%},  Val Loss: {3:>5.2},  Val Acc: {4:>6.2%},  Time: {5} {6}'
                print(msg.format(total_batch, loss.item(), train_acc, dev_loss, dev_acc, time_dif, improve))
                writer.add_scalar("loss/train", loss.item(), total_batch)
                writer.add_scalar("loss/dev", dev_loss, total_batch)
                writer.add_scalar("acc/train", train_acc, total_batch)
                writer.add_scalar("acc/dev", dev_acc, total_batch)
                model.train()
            total_batch += 1
            if total_batch - last_improve > config.require_improvement:
                # 验证集loss超过1000batch没下降，结束训练
                print("No optimization for a long time, auto-stopping...")
                flag = True
                break
        if flag:
            break
    writer.close()
def evaluate(config, model, data_iter, test=False):
    model.eval()
    loss_total = 0
    predict_all = np.array([], dtype=int)
    labels_all = np.array([], dtype=int)
    with torch.no_grad():
        for texts, qinggan_label, zhuti_label in data_iter:
            outputs = model(texts)
            loss = F.cross_entropy(outputs, qinggan_label)
            loss_total += loss
            labels = qinggan_label.data.cpu().numpy()
            predic = torch.max(outputs.data, 1)[1].cpu().numpy()
            labels_all = np.append(labels_all, labels)
            predict_all = np.append(predict_all, predic)

    acc = metrics.accuracy_score(labels_all, predict_all)
    if test:
        report = metrics.classification_report(labels_all, predict_all, target_names=config.class_list, digits=4)
        confusion = metrics.confusion_matrix(labels_all, predict_all)
        return acc, loss_total / len(data_iter), report, confusion
    return acc, loss_total / len(data_iter)
train_qinggan(config, model, train_iter, test_iter)

Epoch [1/10]
Iter:      0,  Train Loss:  0.99,  Train Acc: 58.59%,  Val Loss:   0.9,  Val Acc: 68.68%,  Time: 0:05:39 *improve
Iter:      2,  Train Loss:   1.0,  Train Acc: 56.25%,  Val Loss:  0.89,  Val Acc: 68.68%,  Time: 0:11:37 *improve
Iter:      4,  Train Loss:  0.79,  Train Acc: 73.44%,  Val Loss:  0.87,  Val Acc: 68.68%,  Time: 0:12:05 *improve
Iter:      6,  Train Loss:  0.46,  Train Acc: 91.41%,  Val Loss:   1.0,  Val Acc: 68.68%,  Time: 0:12:30 
Iter:      8,  Train Loss:  0.31,  Train Acc: 92.97%,  Val Loss:   1.3,  Val Acc: 68.68%,  Time: 0:12:55 
Iter:     10,  Train Loss:   2.2,  Train Acc: 42.97%,  Val Loss:   1.1,  Val Acc: 68.68%,  Time: 0:13:20 
Iter:     12,  Train Loss:   1.2,  Train Acc: 60.16%,  Val Loss:   0.9,  Val Acc: 68.68%,  Time: 0:13:46 
Iter:     14,  Train Loss:   1.1,  Train Acc: 53.91%,  Val Loss:  0.91,  Val Acc: 68.15%,  Time: 0:14:10 
Iter:     16,  Train Loss:  0.98,  Train Acc: 63.28%,  Val Loss:   1.0,  Val Acc: 53.60%,  Time: 0:14:37 
Iter:    

In [4]:
class FocalLoss(nn.Module):
    """
    参考 https://github.com/lonePatient/TorchBlocks
    """

    def __init__(self, gamma=2.0, alpha=1, epsilon=1.e-9, device=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        if isinstance(alpha, list):
            self.alpha = torch.Tensor(alpha, device=device)
        else:
            self.alpha = alpha
        self.epsilon = epsilon

    def forward(self, input, target):
        """
        Args:
            input: model's output, shape of [batch_size, num_cls]
            target: ground truth labels, shape of [batch_size]
        Returns:
            shape of [batch_size]
        """
        num_labels = input.size(-1)
        idx = target.view(-1, 1).long()
        one_hot_key = torch.zeros(idx.size(0), num_labels, dtype=torch.float32, device=idx.device)
        one_hot_key = one_hot_key.scatter_(1, idx, 1)
        one_hot_key[:, 0] = 0  # ignore 0 index.
        logits = torch.softmax(input, dim=-1)
        loss = -self.alpha * one_hot_key * torch.pow((1 - logits), self.gamma) * (logits + self.epsilon).log()
        loss = loss.sum(1)
        return loss.mean()



loss = FocalLoss(alpha=[0.1, 0.2, 0.3])
input = torch.randn(5, 3, requires_grad=True)
target = torch.empty(5, dtype=torch.long).random_(3)
output = loss(input, target)
print(output)
output.backward()


tensor(0.0402, grad_fn=<MeanBackward0>)


In [ ]:
optimizer = AdamW(optimizer_grouped_parameters,lr=config.learning_rate,eps=1e-8)

# 学习率指数衰减，每次epoch：学习率 = gamma * 学习率
scheduler = get_linear_schedule_with_warmup(optimizer,
                                        num_warmup_steps = 0,
                                        num_training_steps = len(train_iter) * config.num_epochs)
